In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold
import pandas as pd

In [3]:
df_train = pd.read_csv('dataset/train.csv').fillna('')
df_test = pd.read_csv('dataset/test.csv').fillna('')
# df_sample = pd.read_csv('dataset/sample_submission.csv')

In [7]:
df_train

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,,,6112,598.424000
...,...,...,...,...,...,...
2249693,2422167,Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...,Material : Polyester,,3009,1181.100000
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[❤ [Inspiration] Inspired by the Untitled Goos...,<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,3413,125.984252
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,"[Dial size: 12 inches in diameter,Big, clear r...",Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000
2249696,1165754,Electro Voice EKX-BRKT15 | Wall Mount Bracket ...,,,592,2900.000000


In [4]:
y = df_train['PRODUCT_LENGTH'].astype(int)

X = pd.DataFrame()
X['PRODUCT_TYPE_ID'] = df_train['PRODUCT_TYPE_ID']
X['TEXT'] =  df_train['TITLE'][:1000] + ' ' + df_train['DESCRIPTION'][:1000] + ' ' + df_train['BULLET_POINTS'][:1000]
X = X.to_numpy()

In [5]:
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

In [7]:
transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

# Training config
kfold = KFold(n_splits=7)# StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(X_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 7 folds for each of 20 candidates, totalling 140 fits


ValueError: 
All the 140 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
140 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/software/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/software/anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/software/anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/Users/software/anaconda3/lib/python3.10/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/Users/so

In [ ]:
X_test = pd.DataFrame()
X_test['PRODUCT_TYPE_ID'] = df_test['PRODUCT_TYPE_ID']
X_test['TEXT'] =  df_test['TITLE'] + ' ' + df_test['DESCRIPTION'] + ' ' + df_test['BULLET_POINTS']
X_test = X_test.to_numpy()

y_predicted = rf_model.predict(X_test)

ans = pd.DataFrame()
ans['PRODUCT_ID'] = df_test['PRODUCT_ID']
ans['PRODUCT_LENGTH'] = y_predicted

ans.to_csv('submission_sklearn.csv', index=False)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
from collections import Counter

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop = set(stopwords.words('english'))

X_train_num = [row[:-2].astype(float) for row in X_train]
X_train_text = [row[-1] for row in X_train]

tokenized = []
wnl = nltk.stem.WordNetLemmatizer()
for text in tqdm(X_train_text):
    # Filter alphabet words only and non stop words, make it loser case
    words = [word.lower() for word in word_tokenize(text) if ((word.isalpha()==1) & (word not in stop))]
    # Lemmatize words 
    tokens = [wnl.lemmatize(wnl.lemmatize(word, 'n'), 'v') for word in words]
    tokenized.append(tokens)

all_words = [word for text in tokenized for word in text]
counts = Counter(all_words)
bow = sorted(counts, key=counts.get, reverse=True)
vocab = {word: ii for ii, word in enumerate(counts, 1)}
id2vocab = {v: k for k, v in vocab.items()}
token_ids = [[vocab[word] for word in text_words] for text_words in tokenized]

In [ ]:
# Build a dataloader
def dataloader(text_inputs, num_inputs, labels, sequence_length=200, batch_size=16, shuffle=False):
    if shuffle:
        indices = list(range(len(text_inputs)))
        random.shuffle(indices)
        text_inputs = [text_inputs[idx] for idx in indices]
        num_inputs = [num_inputs[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(text_inputs)

    for ii in range(0, total_sequences, batch_size):
        batch_texts = text_inputs[ii: ii+batch_size]        
        batch = torch.zeros((sequence_length, len(batch_texts)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_texts):
            token_tensor = torch.tensor(tokens)
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        label_tensor = torch.tensor(labels[ii: ii+len(batch_texts)])
        num_tensor = torch.tensor(num_inputs[ii: ii+len(batch_texts)])
        
        yield batch, num_tensor, label_tensor

In [ ]:
from torch import nn

# Model taking non-text and text inputs as well
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, dense_size, num_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(lstm_size, dense_size)
        self.fc2 = nn.Linear(dense_size + num_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """
        Initialize the hidden state
        """
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

    def forward(self, nn_input_text, nn_input_num, hidden_state):
        """
        Perform a forward pass of the model on nn_input
        """
        batch_size = nn_input_text.size(0)
        nn_input_text = nn_input_text.long()
        embeds = self.embedding(nn_input_text)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        # Stack up LSTM outputs, apply dropout
        lstm_out = lstm_out[-1,:,:]
        lstm_out = self.dropout(lstm_out)
        # Dense layer
        dense_out = self.fc1(lstm_out)
        # Concatinate the dense output and meta inputs
        concat_layer = torch.cat((dense_out, nn_input_num.float()), 1)
        out = self.fc2(concat_layer)
        logps = self.softmax(out)

        return logps, hidden_state

In [ ]:
n_input_num = len(X_train_num[0])
y_train = y_train.tolist()

In [ ]:
import torch
from torch import optim

# Training parameters
epochs=3
batch_size=8
learning_rate=1e-4
sequence_length=200
clip=5

# Set model
model = TextClassifier(len(vocab)+1, 512, 128, 8, n_input_num, 3, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
# Set device
device = torch.device('mps')
model.to(device)
# Set Loss Functoin and Optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Start Training
model.train()
for epoch in range(epochs):
    print("###### Epoch {}/{} #####".format(epoch+1, epochs))
    hidden = model.init_hidden(batch_size)
    # Training Step
    for text_batch, mum_batch, labels in tqdm(dataloader(token_ids, X_train_num, y_train, 
            batch_size=batch_size, sequence_length=sequence_length, shuffle=False),
                                              total=int(len(y_train)/batch_size, desc="Training Batch")):
        # Skip the last batch of which size is not equal to batch_size
        if text_batch.size(1) != batch_size:
            break
        # Creating new variables for the hidden state to avoid backprop entire training history
        hidden = tuple([each.data for each in hidden])
        # Set Device
        text_batch, mum_batch, labels = text_batch.to(device), mum_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        model.zero_grad()
        output, hidden = model(text_batch, mum_batch, hidden)
        loss = criterion(output, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

    # Evaluation Step
    # :

In [ ]:
from transformers import BertTokenizer, BertModel

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Model definition
class BertTextClassifier(nn.Module):
    def __init__(self, hidden_size, dense_size, meta_size, output_size, dropout=0.1):
        """
        Initialize the model
        """
        super().__init__()
        self.output_size = output_size
        self.dropout = dropout        
        self.bert = BertModel.from_pretrained('bert-base-uncased',  
                                        output_hidden_states=True,
                                        output_attentions=True)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.weights = nn.Parameter(torch.rand(13, 1))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, dense_size)
        self.fc2 = nn.Linear(dense_size + meta_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, nn_input_meta):
        """
        Perform a forward pass of the model on nn_input
        """
        all_hidden_states, all_attentions = self.bert(input_ids)[-2:]
        batch_size = input_ids.shape[0]
        ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, 768)
        atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
        atten = F.softmax(atten.view(-1), dim=0)
        feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])        
        # Dense layer
        dense_out = self.fc1(self.dropout(feature))
        # Concatinate the dense output and meta inputs
        concat_layer = torch.cat((dense_out, nn_input_meta.float()), 1)
        out = self.fc2(concat_layer)

        return out